In [1]:
from premise import *
import bw2data, bw2io, bw2calc
bw2data.projects.set_current("ecoinvent-3.11-cutoff")

In [2]:
scenarios = [
    {"model": "remind", "pathway": "SSP1-NDC", "year": 2025,},
    #{"model": "remind", "pathway": "SSP1-NPi", "year": 2025,},
    #{"model": "remind", "pathway": "SSP1-PkBudg650", "year": 2025,},
    #{"model": "remind", "pathway": "SSP1-PkBudg1000", "year": 2025,},
    #{"model": "remind", "pathway": "SSP2-rollBack", "year": 2025,},
    #{"model": "remind", "pathway": "SSP2-NDC", "year": 2025,},
    #{"model": "remind", "pathway": "SSP2-NPi", "year": 2025,},
    #{"model": "remind", "pathway": "SSP2-PkBudg650", "year": 2025,},
    #{"model": "remind", "pathway": "SSP2-PkBudg1000", "year": 2025,},
    #{"model": "remind", "pathway": "SSP3-rollBack", "year": 2025,},
    #{"model": "remind", "pathway": "SSP3-NDC", "year": 2025,},
    #{"model": "remind", "pathway": "SSP3-NPi", "year": 2025,},
    #{"model": "remind-eu", "pathway": "SSP2-NDC", "year": 2025,},
    #{"model": "remind", "pathway": "SSP3-PkBudg1000", "year": 2025,},
    #{"model": "remind-eu", "pathway": "SSP2-NPi", "year": 2025,},
    #{"model": "remind-eu", "pathway": "SSP2-PkBudg650", "year": 2025,},
    #{"model": "remind-eu", "pathway": "SSP2-PkBudg1000", "year": 2025,},

    #{"model": "image", "pathway": "SSP1-L", "year": 2025,},
    #{"model": "image", "pathway": "SSP1-Ma", "year": 2025,},
    #{"model": "image", "pathway": "SSP1-VLLO", "year": 2025,},
    #{"model": "image", "pathway": "SSP2-L", "year": 2025,},
    #{"model": "image", "pathway": "SSP2-M", "year": 2025,},
    #{"model": "image", "pathway": "SSP2-VLHO", "year": 2025,},
    #{"model": "image", "pathway": "SSP5-H", "year": 2025,},
    #{"model": "image", "pathway": "SSP3-H", "year": 2025,},

    #{"model": "tiam-ucl", "pathway": "SSP2-Base", "year": 2025,},
    #{"model": "tiam-ucl", "pathway": "SSP2-RCP19", "year": 2025,},
    #{"model": "tiam-ucl", "pathway": "SSP2-RCP26", "year": 2025,},
    #{"model": "tiam-ucl", "pathway": "SSP2-RCP45", "year": 2025,},
]


ndb = NewDatabase(
        scenarios = scenarios,
        source_db="ecoinvent-3.11-cutoff",
        source_version="3.11",
        key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',
)


premise v.(2, 3, 0, 'dev1')
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your Brightway project:                                   |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-----

In [3]:
ndb.generate_scenario_report()

Generate scenario report.
Report saved under /Users/romain/GitHub/premise/dev/export/scenario_report.


In [6]:
import bw2data, bw2io, bw2calc
bw2data.projects.set_current("ei38_2")

In [4]:
[(f["name"], f["code"]) for f in bw2data.Database("biosphere3") if f["name"] == "Fish, pelagic, in ocean"]

[('Fish, pelagic, in ocean', 'd22719d0-8afd-45d3-b9eb-222112f84cc7')]

In [17]:
import pandas as pd

main_df = pd.DataFrame()
for scenario in ndb.scenarios:
    df = scenario["iam data"].other_vars.sel(variables="GMST", region="World").to_dataframe("x")
    df["model"] = scenario["model"]
    df["scenario"] = scenario["pathway"]
    main_df = pd.concat([main_df, df])

In [19]:
main_df.to_excel("temps.xlsx")

In [18]:
main_df

,region,variables,x,model,scenario
year,,,,,
2005,World,GMST,0.972309,remind,SSP1-NDC
2010,World,GMST,1.074420,remind,SSP1-NDC
2015,World,GMST,1.209518,remind,SSP1-NDC
2020,World,GMST,1.319732,remind,SSP1-NDC
2025,World,GMST,1.467861,remind,SSP1-NDC
...,...,...,...,...,...
2060,World,GMST,2.075522,tiam-ucl,SSP2-RCP45
2070,World,GMST,2.258184,tiam-ucl,SSP2-RCP45
2080,World,GMST,2.435781,tiam-ucl,SSP2-RCP45
